In [1]:
import scipy
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt

In [168]:
# parameters
json_path = 'setting/C0016.json'
pattern_size = (15, 15)
square_size = 1
output_path = 'coords_voltages/coords_voltages_C0016.mat'
detection_path = 'detection_result/detection_result_C0016.mat'

f = open(json_path, 'r')
content = json.load(f)
voltage_path = content['voltage_path']
wall_board_time = content['wall_board_time']
spad_board_time = content['spad_board_time']
video_path = content['video_path']
time = wall_board_time
video = cv2.VideoCapture(video_path)

In [169]:
def get_video_frame(video, time):
    time *= 1000
    video.set(cv2.CAP_PROP_POS_MSEC, time) # gray scale image
    success, image = video.read()
    if success:
        return image
    return None

img = get_video_frame(video, time)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [184]:
imgpoints = []
objpoints = []

# Find the chess board corners
ret, corners = cv2.findChessboardCorners(img, (15, 15), None)

# If found, add object points, image points (after refining them)
if ret == True:
    cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
    imgpoints.append(corners)
    imgpoints = np.array(imgpoints)
    imgpoints_squeezed = np.squeeze(imgpoints)

In [185]:
objp = np.zeros((15*15, 3), np.float32)
objp[:, :2] = np.mgrid[0:15, 0:15].T.reshape(-1, 2)
objpoints.append(objp)
objpoints_squeezed = np.squeeze(np.array(objpoints))

retval, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [259]:
def img_to_world(img_coords, rvecs, tvecs, camera_matrix):
    
    """
    img_coords: (num_of_points, 2)
    rvecs, tvecs: tuple
    """

    img_coords = img_coords.T # (2, num_of_points) 
    img_coords = np.concatenate((img_coords, np.ones([1, img_coords.shape[1]])), axis=0) # (3, num_of_points) 
    
    # Transfer rotation vector to rotation matrix, and get translation matrix from tuple.
    R, _ = cv2.Rodrigues(rvecs[0])
    t = tvecs[0]

    extrinsic = np.concatenate((R, t), axis=1)
    img_coords = camera_matrix @ extrinsic @ img_coords
    img_coords = img_coords / img_coords[2, :]
    
    return world_coords[:3, :] # (3, num_of_points)


In [257]:
def world_to_img(world_coords, rvecs, tvecs, camera_matrix):
    
    """
    world_coords: (num_of_points, 3)
    rvecs, tvecs: tuple
    """
    
    world_coords = world_coords.T # (3, num_of_points) 
    world_coords = np.concatenate((world_coords, np.ones([1, world_coords.shape[1]])), axis=0) # (4, num_of_points) 
    
    # Transfer rotation vector to rotation matrix, and get translation matrix from tuple.
    R, _ = cv2.Rodrigues(rvecs[0])
    t = tvecs[0]
    
    extrinsic = np.concatenate((R, t), axis=1)
    img_coords = camera_matrix @ extrinsic @ world_coords
    img_coords = img_coords / img_coords[2, :]

    return img_coords[:2, :] # (2, num_of_points)


In [228]:
projected_img_points, _ = cv2.projectPoints(np.array(objpoints), rvecs[0], tvecs[0], camera_matrix, dist_coeffs)

In [258]:
# img_to_world(imgpoints_squeezed, rvecs, tvecs, camera_matrix)
world_to_img(objpoints_squeezed, rvecs, tvecs, camera_matrix)[:, 3]

array([2.06260907e+03, 4.97012408e+02, 1.00000000e+00])

In [201]:
projected_img_points

array([[[2059.824  ,  402.1954 ]],

       [[2060.9287 ,  433.20453]],

       [[2062.029  ,  464.0878 ]],

       [[2063.1252 ,  494.8437 ]],

       [[2064.218  ,  525.4714 ]],

       [[2065.3074 ,  555.97076]],

       [[2066.3936 ,  586.3421 ]],

       [[2067.4768 ,  616.5862 ]],

       [[2068.5576 ,  646.70404]],

       [[2069.6355 ,  676.6967 ]],

       [[2070.7112 ,  706.5656 ]],

       [[2071.7842 ,  736.3122 ]],

       [[2072.8547 ,  765.93787]],

       [[2073.9229 ,  795.4441 ]],

       [[2074.9885 ,  824.8323 ]],

       [[2035.314  ,  405.78494]],

       [[2036.4729 ,  436.9086 ]],

       [[2037.6273 ,  467.90482]],

       [[2038.7776 ,  498.77213]],

       [[2039.9241 ,  529.50995]],

       [[2041.0669 ,  560.1182 ]],

       [[2042.2064 ,  590.5974 ]],

       [[2043.3427 ,  620.94824]],

       [[2044.476  ,  651.1718 ]],

       [[2045.6062 ,  681.2694 ]],

       [[2046.7335 ,  711.2424 ]],

       [[2047.8579 ,  741.0922 ]],

       [[2048.9795 ,  770.82